In [6]:
import pyttsx3

engine = pyttsx3.init()

engine.setProperty('rate', 150)

engine.setProperty('volume',0.9)
voices = engine.getProperty('voices')
engine.setProperty('voice',voices[1].id)

text = "Hello, I am Butkai. I am your voice assistant"

engine.say(text)

engine.runAndWait()

In [21]:
import speech_recognition as sr 
import pyttsx3
from datetime import datetime
import shutil
import os
import glob
import webbrowser
import pyautogui
import cv2
import time
import subprocess
import winreg as reg
import re
import spacy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv


# Initializing the text to speech engine
engine = pyttsx3.init()
engine.setProperty('rate', 150)
engine.setProperty('volume',0.9)
voices = engine.getProperty('voices')
engine.setProperty('voice',voices[1].id)

def speak(text):
    """Convert text to speech."""
    engine.say(text)
    engine.runAndWait()

def wishMe():
    """Wish the user based on the current time."""
    hour = int(datetime.now().hour)
    if hour >= 0 and hour < 12:
        speak("Good Morning!")
    elif hour >= 12 and hour < 18:
        speak("Good Afternoon!")
    else:
        speak("Good Evening!")
    speak("I am Butkai, your voice assistant. How can I assist you today?")

def listen(prompt=None):
    """Listen to user's voice command with optional prompt"""
    if prompt:
        speak(prompt)
    
    with sr.Microphone() as source:
        r = sr.Recognizer()
        r.adjust_for_ambient_noise(source,duration=0.5)
        r.pause_threshold, r.energy_threshold = 1, 4000
        print("Listening..." if not prompt else f"Listening for: {prompt}")
        try:
            audio = r.listen(source, timeout=5)
            command = r.recognize_google(audio).lower()
            print(f"User: {command}")
            return command
        except sr.WaitTimeoutError:
            speak("I didn't hear anything.")
            return None
        except sr.UnknownValueError:
            speak("Sorry, I didn't understand that.")
            return None
        except sr.RequestError:
            speak("Speech service unavailable.")
            return None

def userName():
    """Get the user's name using the listen function"""
    max_attempts = 3
    for attempt in range(max_attempts):
        uname = listen("What is your name?" if attempt == 0 else "Could you please repeat your name?")
        
        if uname:
            uname = uname.strip()
            columns = shutil.get_terminal_size().columns
            print("#####################".center(columns))
            print("Welcome ", uname.center(columns))
            print("#####################".center(columns))
            return uname
        
        if attempt < max_attempts - 1:
            speak(f"Attempt {attempt + 2} of {max_attempts}")
    
    speak("I'll call you friend for now.")
    return "friend"

def take_selfie():
    """Take a selfie using the webcam"""
    speak("Taking a selfie in 3 seconds. Smile!")
    time.sleep(1)
    speak("3")
    time.sleep(1)
    speak("2")
    time.sleep(1)
    speak("1")
    
    # Initialize the webcam
    cap = cv2.VideoCapture(0)
    
    # Check if the webcam is opened correctly
    if not cap.isOpened():
        speak("I couldn't access the camera.")
        return
    
    ret, frame = cap.read()
    
    if ret:
        # Create a directory for selfies if it doesn't exist
        if not os.path.exists('selfies'):
            os.makedirs('selfies')
        
        # Save the selfie with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"selfies/selfie_{timestamp}.jpg"
        cv2.imwrite(filename, frame)
        speak("Selfie captured successfully!")
    else:
        speak("Oops! Something went wrong while taking the selfie.")
    
    # Release the camera
    cap.release()

def get_spotify_path():
    """Search in windows registry for spotify path"""
    common_paths = [
        os.path.expandvars("%APPDATA%\\Spotify\\Spotify.exe"),
        os.path.expandvars("%LOCALAPPDATA%\\Microsoft\\WindowsApps\\Spotify.exe"),
        "C:\\Program Files\\Spotify\\Spotify.exe",
        "C:\\Program Files (x86)\\Spotify\\Spotify.exe",
        os.path.expandvars("%USERPROFILE%\\AppData\\Local\\Microsoft\\WindowsApps\\Spotify.exe"),
    ]

    # Checking each path if exists
    for path in common_paths:
        if os.path.exists(path):
            return path
        
    # Checking in registry
    try:
        with reg.OpenKey(reg.HKEY_LOCAL_MACHINE, r"SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Uninstall\\Spotify") as key:
            install_location = reg.QueryValueEx(key, "InstallLocation")[0]
            spotify_path = os.path.join(install_location, "Spotify.exe")
            if os.path.exists(spotify_path):
                return spotify_path
    except WindowsError:
        pass
    except FileNotFoundError:
        print("Spotify not found in registry.")

    # Try searching in program files
    program_files_dirs = [
        os.environ.get("ProgramFiles"),
        os.environ.get("ProgramFiles(x86)"),
        os.environ.get("LOCALAPPDATA"),
        os.environ.get("APPDATA"),
    ]
        
    for dir in program_files_dirs:
        if not dir:
            continue
        for match in glob.glob(os.path.join(dir, "**", "Spotify.exe"), recursive=True):
            if os.path.exists(match):
                return match
    
    speak("Spotify not found. Please install Spotify or check your installation.")
    return None

def get_song_name(song_name):
    """Extract song name from the command"""
    noise_phrases = ["on spotify", "please", "could you", "hey assistant","in spotify"]
    # Clean the command by removing noise phrases
    clean_command = song_name.lower()
    for phrase in noise_phrases:
        clean_command = clean_command.replace(phrase, "")
    
    play_triggers = r"(play|listen to|stream|queue)\s*(?:the)?\s*"
    match = re.search(f"{play_triggers}(.*)", clean_command)
    
    if match:
        song_part = match.group(2)
        # Remove artist clauses if needed
        return re.sub(r"\s*(by|from|ft\.?)\s+.*", "", song_part).strip()
    
    return song_name

def get_song_info(song_name):
    load_dotenv()
    client_id = os.getenv("SPOTIPY_CLIENT_ID")
    client_secret = os.getenv("SPOTIPY_CLIENT_SECRET")
    # Authenticate with Spotify
    auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(auth_manager=auth_manager)

    # Search for the track
    result = sp.search(q=song_name, type='track', limit=1)
    tracks = result.get('tracks', {}).get('items', [])

    if not tracks:
        return f"No results found for '{song_name}'."

    # Extract track details
    track = tracks[0]
    title = track['name']
    artists = ', '.join([artist['name'] for artist in track['artists']])
    duration_ms = track['duration_ms']
    duration_sec = duration_ms // 1000  # Convert ms to seconds

    return {
        "title": title,
        "artist": artists,
        "duration_seconds": duration_sec
    }

def open_spotify_play(song_name, uname):
    """Open spotify app and play user requested song"""
    song = get_song_name(song_name)
    print(song)
    song_info = get_song_info(song)
    speak(f'Playing {song} by {song_info.get("artist", None)}')
    
    try:
        spotify_path = get_spotify_path()
        subprocess.Popen(spotify_path)
        time.sleep(2)
    except Exception as e:
        webbrowser.open("https://open.spotify.com")
        time.sleep(5) # Wait for 5 sec for webpage to load
        speak(f"{uname} Spotify not found. Opening Spotify website instead.")
    
    pyautogui.hotkey('ctrl', 'l')  # Focus on the search bar
    time.sleep(0.5)
    pyautogui.write(song, interval=0.1)
    pyautogui.press('enter')
    time.sleep(1)
    pyautogui.press('tab', presses=4, interval=0.2)
    pyautogui.press('space')
    
    song_time = song_info.get("duration_seconds", 0)
    remaining_time = song_time
    song_timer = wait_and_listen(song_time, remaining_time)  # Wait for the song to play for its duration
    return song_timer

def wait_and_listen(song_time, remaining_time):
    """It will dynamically adjust the sleep time based on the user's command"""
    start_time = time.time()
    while remaining_time > 0:
        remaining_time = song_time - int(time.time() - start_time)
        command = listen()
        if command:
            if 'pause' in command or 'stop' in command:
                speak("Pausing the song.")
                spotify_path = get_spotify_path()
                subprocess.Popen(spotify_path)
                pyautogui.press('space')
                song_timer = remaining_time
                
            elif 'resume' in command or 'continue' in command:
                speak("Resuming the song.")
                pyautogui.press('space')
                time.sleep(1)
                remaining_time = wait_and_listen(song_timer, remaining_time)
    return remaining_time


def process_command(command, uname):
    """Process user commands"""
    if 'spotify' in command and 'play' in command:
        # Extract song name
        song_start = command.find('play') + 5
        song_name = command[song_start:]
        open_spotify_play(song_name, uname)
    elif 'selfie' in command or 'photo' in command:
        take_selfie()
    elif 'exit' in command or 'quit' in command or 'stop' in command:
        speak(f"Goodbye! {uname}. Have a great day!")
        exit()
    else:
        speak(f"{uname} I didn't understand that command. Can you try again?")

def main():
    os.system('cls' if os.name == 'nt' else 'clear')
    wishMe()
    uname = userName()
    speak(f"Welcome {uname}")
    speak("How can I assist you today?")
    
    while True:
        command = listen()
        if not command:
            continue
            
        # Check for exit commands
        if any(word in command for word in ["quit", "stop", "exit", "goodbye"]):
            speak(f"Goodbye {uname}! Have a nice day.")
            break
            
        process_command(command, uname)

In [20]:
if __name__ == "__main__":
    main()

Listening for: What is your name?
Listening for: Could you please repeat your name?
User: ajay
                             #####################                              
Welcome                                        ajay                                      
                             #####################                              
Listening...
User: play we need love on spotify
we need love on spotify
Listening...
User: pause the song
Listening...
User: resume the song
Listening...
User: resume
Listening...
User: resume
Listening...
User: take a selfie
Listening...
User: exit


In [ ]:
open_spotify_play("Play we found love on spotify","Ajay")

we found love
Listening...
User: pause
Listening...
User: resume
Listening...
User: pause
Listening...
User: take a selfie
Listening...
User: take a selfie
Listening...
Listening...
User: take a selfie
Listening...


Exception ignored in: <function BSTR.__del__ at 0x000002865BBD93A0>
Traceback (most recent call last):
  File "c:\Users\sayan\anaconda3\envs\voice-chatbot-env\lib\site-packages\comtypes\_post_coinit\bstr.py", line 26, in __del__
    def __del__(self, _free: Callable[["BSTR"], Any] = _SysFreeString) -> None:
KeyboardInterrupt: 


Listening...
Listening...


In [19]:
get_song_name("Play we need love on spotify")

'we need love'

In [8]:
song = get_song_name("Play we found love on spotify")
print(song)
song_info = get_song_info(song)
print(song_info)

we found love
{'title': 'We Found Love', 'artist(s)': 'Rihanna, Calvin Harris', 'duration_seconds': 215}


In [2]:
from dotenv import load_dotenv
import os

os.getenv("SPOTIPY_CLIENT_SECRET")

'283edda3bb974a4d8818624e97927cd3'

In [ ]:
import speech_recognition as sr 
import pyttsx3
from datetime import datetime
import shutil
import os
import glob
import webbrowser
import pyautogui
import cv2
import time
import subprocess
import winreg as reg
import re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv

# Global state variables
current_song = None
is_paused = False
song_timer = 0

# Initializing the text to speech engine
engine = pyttsx3.init()
engine.setProperty('rate', 150)
engine.setProperty('volume', 0.9)
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

def speak(text):
    """Convert text to speech."""
    engine.say(text)
    engine.runAndWait()

def wishMe():
    """Wish the user based on the current time."""
    hour = int(datetime.now().hour)
    if hour >= 0 and hour < 12:
        speak("Good Morning!")
    elif hour >= 12 and hour < 18:
        speak("Good Afternoon!")
    else:
        speak("Good Evening!")
    speak("I am Butkai, your voice assistant. How can I assist you today?")

def listen(prompt=None, timeout=5):
    """Listen to user's voice command with optional prompt"""
    if prompt:
        speak(prompt)
    
    with sr.Microphone() as source:
        r = sr.Recognizer()
        r.adjust_for_ambient_noise(source, duration=0.5)
        r.pause_threshold, r.energy_threshold = 1, 4000
        print("Listening..." if not prompt else f"Listening for: {prompt}")
        try:
            audio = r.listen(source, timeout=timeout)
            command = r.recognize_google(audio).lower()
            print(f"User: {command}")
            return command
        except sr.WaitTimeoutError:
            speak("I didn't hear anything.")
            return None
        except sr.UnknownValueError:
            speak("Sorry, I didn't understand that.")
            return None
        except sr.RequestError:
            speak("Speech service unavailable.")
            return None

def userName():
    """Get the user's name using the listen function"""
    max_attempts = 3
    for attempt in range(max_attempts):
        uname = listen("What is your name?" if attempt == 0 else "Could you please repeat your name?")
        
        if uname:
            uname = uname.strip()
            columns = shutil.get_terminal_size().columns
            print("#####################".center(columns))
            print("Welcome ", uname.center(columns))
            print("#####################".center(columns))
            return uname
        
        if attempt < max_attempts - 1:
            speak(f"Attempt {attempt + 2} of {max_attempts}")
    
    speak("I'll call you friend for now.")
    return "friend"

def take_selfie():
    """Take a selfie using the webcam"""
    global is_paused
    
    # Temporarily minimize Spotify if it's playing
    if current_song and not is_paused:
        pyautogui.hotkey('alt', 'tab')  # Switch to Spotify
        pyautogui.press('space')  # Pause
        is_paused = True
        time.sleep(0.5)
    
    speak("Taking a selfie in 3 seconds. Smile!")
    time.sleep(1)
    speak("3")
    time.sleep(1)
    speak("2")
    time.sleep(1)
    speak("1")
    
    # Initialize the webcam
    cap = cv2.VideoCapture(0)
    
    # Check if the webcam is opened correctly
    if not cap.isOpened():
        speak("I couldn't access the camera.")
        return
    
    ret, frame = cap.read()
    
    if ret:
        # Create a directory for selfies if it doesn't exist
        if not os.path.exists('selfies'):
            os.makedirs('selfies')
        
        # Save the selfie with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"selfies/selfie_{timestamp}.jpg"
        cv2.imwrite(filename, frame)
        speak("Selfie captured successfully!")
    else:
        speak("Oops! Something went wrong while taking the selfie.")
    
    # Release the camera
    cap.release()
    
    # Restore music playback if it was paused for the selfie
    if current_song and is_paused:
        pyautogui.hotkey('alt', 'tab')  # Switch back to Spotify
        pyautogui.press('space')  # Play
        is_paused = False
        time.sleep(0.5)

def get_spotify_path():
    """Search in windows registry for spotify path"""
    common_paths = [
        os.path.expandvars("%APPDATA%\\Spotify\\Spotify.exe"),
        os.path.expandvars("%LOCALAPPDATA%\\Microsoft\\WindowsApps\\Spotify.exe"),
        "C:\\Program Files\\Spotify\\Spotify.exe",
        "C:\\Program Files (x86)\\Spotify\\Spotify.exe",
        os.path.expandvars("%USERPROFILE%\\AppData\\Local\\Microsoft\\WindowsApps\\Spotify.exe"),
    ]

    # Checking each path if exists
    for path in common_paths:
        if os.path.exists(path):
            return path
        
    # Checking in registry
    try:
        with reg.OpenKey(reg.HKEY_LOCAL_MACHINE, r"SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Uninstall\\Spotify") as key:
            install_location = reg.QueryValueEx(key, "InstallLocation")[0]
            spotify_path = os.path.join(install_location, "Spotify.exe")
            if os.path.exists(spotify_path):
                return spotify_path
    except WindowsError:
        pass
    except FileNotFoundError:
        print("Spotify not found in registry.")

    # Try searching in program files
    program_files_dirs = [
        os.environ.get("ProgramFiles"),
        os.environ.get("ProgramFiles(x86)"),
        os.environ.get("LOCALAPPDATA"),
        os.environ.get("APPDATA"),
    ]
        
    for dir in program_files_dirs:
        if not dir:
            continue
        for match in glob.glob(os.path.join(dir, "**", "Spotify.exe"), recursive=True):
            if os.path.exists(match):
                return match
    
    speak("Spotify not found. Please install Spotify or check your installation.")
    return None

def get_song_name(song_name):
    """Extract song name from the command"""
    noise_phrases = ["on spotify", "please", "could you", "hey assistant", "in spotify"]
    clean_command = song_name.lower()
    for phrase in noise_phrases:
        clean_command = clean_command.replace(phrase, "")
    
    play_triggers = r"(play|listen to|stream|queue)\s*(?:the)?\s*"
    match = re.search(f"{play_triggers}(.*)", clean_command)
    
    if match:
        song_part = match.group(2)
        # Remove artist clauses if needed
        return re.sub(r"\s*(by|from|ft\.?)\s+.*", "", song_part).strip()
    
    return song_name

def get_song_info(song_name):
    load_dotenv()
    client_id = os.getenv("SPOTIPY_CLIENT_ID")
    client_secret = os.getenv("SPOTIPY_CLIENT_SECRET")
    # Authenticate with Spotify
    auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(auth_manager=auth_manager)

    # Search for the track
    result = sp.search(q=song_name, type='track', limit=1)
    tracks = result.get('tracks', {}).get('items', [])

    if not tracks:
        return None

    # Extract track details
    track = tracks[0]
    title = track['name']
    artists = ', '.join([artist['name'] for artist in track['artists']])
    duration_ms = track['duration_ms']
    duration_sec = duration_ms // 1000  # Convert ms to seconds

    return {
        "title": title,
        "artist": artists,
        "duration_seconds": duration_sec
    }

def open_spotify_play(song_name, uname):
    """Open spotify app and play user requested song"""
    global current_song, is_paused, song_timer
    
    song = get_song_name(song_name)
    song_info = get_song_info(song)
    
    if not song_info:
        speak(f"Sorry {uname}, I couldn't find that song.")
        return
    
    current_song = song_info
    is_paused = False
    speak(f'Playing {song_info["title"]} by {song_info["artist"]}')
    
    try:
        spotify_path = get_spotify_path()
        subprocess.Popen(spotify_path)
        time.sleep(2)
    except Exception as e:
        webbrowser.open("https://open.spotify.com")
        time.sleep(5)
        speak(f"{uname} Spotify not found. Opening Spotify website instead.")
    
    pyautogui.hotkey('ctrl', 'l')  # Focus on search bar
    time.sleep(0.5)
    pyautogui.write(song, interval=0.1)
    pyautogui.press('enter')
    time.sleep(1)
    pyautogui.press('tab', presses=4, interval=0.2)
    pyautogui.press('space')
    
    song_timer = song_info["duration_seconds"]
    monitor_playback(uname)

def monitor_playback(uname):
    """Monitor playback and handle commands"""
    global is_paused, song_timer
    
    start_time = time.time()
    remaining_time = song_timer
    
    while remaining_time > 0:
        if not is_paused:
            remaining_time = song_timer - (time.time() - start_time)
            print(f"\rTime remaining: {int(remaining_time)} seconds", end="")
        
        command = listen(timeout=1)
        if command:
            if 'pause' in command and not is_paused:
                speak("Pausing the song.")
                pyautogui.press('space')
                is_paused = True
                song_timer = remaining_time
            elif 'resume' in command and is_paused:
                speak("Resuming the song.")
                pyautogui.press('space')
                is_paused = False
                start_time = time.time()  # Reset timer
            elif 'selfie' in command or 'photo' in command:
                take_selfie()
            elif 'exit' in command or 'quit' in command:
                speak("Stopping playback.")
                return
        
        time.sleep(0.1)

def process_command(command, uname):
    """Process user commands"""
    global current_song, is_paused
    
    if 'spotify' in command and 'play' in command:
        song_start = command.find('play') + 5
        song_name = command[song_start:]
        open_spotify_play(song_name, uname)
    elif 'selfie' in command or 'photo' in command:
        take_selfie()
    elif 'pause' in command and current_song and not is_paused:
        speak("Pausing the song.")
        pyautogui.press('space')
        is_paused = True
    elif 'resume' in command and current_song and is_paused:
        speak("Resuming the song.")
        pyautogui.press('space')
        is_paused = False
    elif 'exit' in command or 'quit' in command or 'stop' in command:
        speak(f"Goodbye! {uname}. Have a great day!")
        exit()
    else:
        speak(f"{uname} I didn't understand that command. Can you try again?")

def main():
    os.system('cls' if os.name == 'nt' else 'clear')
    wishMe()
    uname = userName()
    speak(f"Welcome {uname}")
    speak("How can I assist you today?")
    
    while True:
        command = listen()
        if not command:
            continue
            
        if any(word in command for word in ["quit", "stop", "exit", "goodbye"]):
            speak(f"Goodbye {uname}! Have a nice day.")
            break
            
        process_command(command, uname)

if __name__ == "__main__":
    main()

Listening for: What is your name?
Listening for: Could you please repeat your name?
User: ajay
                             #####################                              
Welcome                                        ajay                                      
                             #####################                              
Listening...
User: play we need love on spotify
Time remaining: 334 secondsListening...
Time remaining: 330 secondsListening...
Time remaining: 326 secondsListening...
Time remaining: 322 secondsListening...
Time remaining: 318 secondsListening...
Time remaining: 314 secondsListening...
User: pause pause pause pause
Listening...
User: take a selfie
Time remaining: 267 secondsListening...
Time remaining: 263 secondsListening...
Time remaining: 259 secondsListening...
Time remaining: 255 secondsListening...
Time remaining: 251 secondsListening...
Time remaining: 247 secondsListening...


Exception ignored in: <function BSTR.__del__ at 0x00000250985138B0>
Traceback (most recent call last):
  File "c:\Users\sayan\anaconda3\envs\voice-chatbot-env\lib\site-packages\comtypes\_post_coinit\bstr.py", line 26, in __del__
    def __del__(self, _free: Callable[["BSTR"], Any] = _SysFreeString) -> None:
KeyboardInterrupt: 


Time remaining: 243 secondsListening...
Time remaining: 239 secondsListening...
Time remaining: 235 secondsListening...
